# Post-Retriever: How to effectively use retrieved knowledge
1. Reranking retrived results
    * ACS Semantic Reranking:  https://learn.microsoft.com/azure/search/semantic-search-overview
    * LLM as Re-Ranking Agent: https://github.com/sunnweiwei/RankGPT
    * Some require special model to Make Reranking: zephyr
    * Weighted score by time, graphic ...
2. Reorder :  best performance typically arises when crucial data is positioned at the start or conclusion of the input context. https://arxiv.org/abs/2307.03172
3. Filter:
    * by date
    * by similarity
    * by reranking score
    * by keywords
4. Context Compression (LongLLMLingua): introduce compression model (SLM) to extract keyword in context. Hard to read by human by can understand by LLM

# Generate Answer
* Summarize by visiting retrieved information chunk one by one
* Summarize by Concat all retrieved information chunk in llm call
* Summarize by tree. When deal with very big information chunks, it concatenate the chunk as much as possible to fit the chunk windows
* Concat answer for each information chunk

In [1]:
from azureresource import (
    get_llm,
    get_embed_model,
    get_vector_store
)
from index import get_index

index_dict = {}
llm = get_llm("gpt-35-turbo", "gpt-35-turbo-1106")
embed_model = get_embed_model("text-embedding-ada-002", "text-embedding-ada-002")
vector_store = get_vector_store("chunk-512")
index = get_index(vector_store, llm, embed_model)


/Users/zhuzhirui/vscodeworkspace/Travel/.venv/lib/python3.11/site-packages/ebooklib/epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


In [6]:
from llama_index.core.vector_stores.types import VectorStoreQueryMode

hybrid_retriever = index.as_retriever(
    vector_store_query_mode=VectorStoreQueryMode.HYBRID,
    similarity_top_k=5
)

basic_retriever = index.as_retriever(
    similarity_top_k=5
)

from testcase import question_list, bcolors

for q in question_list:
    print(f"{bcolors.BOLD}{bcolors.HEADER}Q: {q}{bcolors.ENDC}")
    print(f"{bcolors.HEADER}basic_retriever{bcolors.ENDC}")
    results = basic_retriever.retrieve(q)
    for node in results[:5]:
        print(f"{bcolors.BOLD}Node: {node.node_id}\tScore: {node.score}{bcolors.ENDC}")
        print(node.text)

    print(f"{bcolors.HEADER}retriever with semantic rerank{bcolors.ENDC}")
    results = hybrid_retriever.retrieve(q)
    for node in results[:5]:
        print(f"{bcolors.BOLD}Node: {node.node_id}\tScore: {node.score}{bcolors.ENDC}")
        print(node.text)

Q: 在巴塞罗那可以体验哪些有趣的事情
basic_retriever
Node: 267d6dd1-a64f-4ad6-bc27-1fbe45621763	Score: 0.8765443
二十世纪巴塞罗那的很多故事发生在这个广场上。

![image](./Images/004.jpeg)

享受街头喝啤酒的美国年轻人（这在美国是违禁的）

我们两次来巴塞罗那，都住过老城。这次，我们拖着行李从大道的起点，顺着步行道的中间，走了整整两站地铁的距离，只是为了感受一下这里的欢乐。一个小酒铺前，桌子拼在一起，变成长长的一溜。周围坐了一圈兴高采烈的美国大男孩，看到他们，我们忍不住会心笑起来，因为他们每人手里端着一个特大号的啤酒杯。照美国的法律，在大街上这么喝酒是不容许的。他们兴奋不已，只是因为他们可以感受“合法地在大街上喝酒”的刺激，再说，还是以那么开心的方式。我们说，让我们拍张照啊，他们很助兴，齐齐地高高举起了冒着泡沫的大玻璃杯。我们一路走一路给艺人们打分，最后一致同意，“最佳创意”颁给了两个和骷髅一起骑自行车的艺人。
Node: fddc9a87-9996-4b99-9f2d-383763b19552	Score: 0.87324184
花一天青年旅舍的钱，在这里可以住三天。但我们在巴塞罗那错失的青年旅舍的经验，终于后来在塞维利亚补上了。青年旅舍现在有统间，也有双人单间，可以在网上预订，也可以电话预订。它遍布世界，实在是很方便穷旅人。有关青年旅舍，还有一个朋友的经验是，她遇到过陌生男女同住的大统间。一开始真是吓了一跳。

转到那个不留心去记住、下次就要找不到的小街，顿时感觉很歉疚，相信找到这个地方，真是花费了朋友很多精力。那是巴塞罗那主教堂附近的哥特区，拉布拉斯大道边上，是老城的小巷。对旅游者来说，位置极好。你因此可以在老城区逗留得很晚，回住处很方便。

那是真正的老巴塞罗那。可是，有了这次的经验，才知道那一片已经变成穷人聚集的地方。其实，这和所有的大都市是一样的。纽约、巴黎也有大片如此的老城区。那都是旧房，至少有上几百年的历史了。这些房子要修的话，耗费巨大，而且旧的结构在那里，改造也相当困难。
Node: 94196ab4-8b43-4a21-b16a-0fcb10d5288c	Score: 0.8729602
旅人们到西班牙，首都马德里当是首选，这是西班牙在地理上